In [17]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.transformer import Transformer
from sagemaker.xgboost import XGBoostModel
import boto3

SETTINGS

In [18]:
# Define the variables (replace these with your actual values)
model_name ="my-xgboost-model-from-training-job"  # The name of your existing model
transform_job_name = 'my-fancy-name'
input_data_path = 's3://sagemaker-bucket-ds/training-jobs/data/inference_input/'
output_data_path = 's3://sagemaker-bucket-ds/training-jobs/data/inference_output/'
instance_type = 'ml.m5.large'  # Replace with your preferred instance type

DELETE THE OUTPUT

In [19]:
!aws s3 rm s3://sagemaker-bucket-ds/training-jobs/data/inference_output/ --recursive

BASIC OBJECTS

In [20]:
role = get_execution_role()  # You can also specify the role ARN directly
sagemaker_session = sagemaker.Session()

CREATE TRANSFORMER

In [21]:
# Create the Transformer object directly using the existing model name
transformer = Transformer(
    model_name=model_name,
    instance_count=1,
    instance_type=instance_type,
    output_path=output_data_path,
    assemble_with='Line',
    accept='text/csv'
)

MAKE TRANSFORMATION

In [22]:
# Start the batch transform job
transformer.transform(
    data=input_data_path,
    content_type='text/csv',
    split_type='Line'
)

# Wait for the transform job to complete
transformer.wait()

print("Transform job completed. Output stored in:", output_data_path)

INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2024-07-01-12-32-33-606


.............................[2024-07-01:12:37:25:INFO] No GPUs detected (normal if no gpus installed)
[2024-07-01:12:37:25:INFO] No GPUs detected (normal if no gpus installed)
[2024-07-01:12:37:25:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invocations|execution-parameters) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_read_timeout 60s;
      proxy_pass http://gunicorn;
    }
    location / {
      return 404 "{}";
    }
  }
}
[2024-07-01 12:37:25 +0000] [16] [INFO] Starting gunico

MAKE TRANSFORMATION